<a href="https://colab.research.google.com/github/visiont3lab/mask-detector-covid/blob/main/notebooks/Project_Covid_Mask_Classifier_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Get data
!git clone https://github.com/visiont3lab/mask-detector-covid.git

Cloning into 'mask-detector-covid'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 83 (delta 43), reused 37 (delta 18), pack-reused 0
Unpacking objects: 100% (83/83), done.


In [ ]:
# Haar cascade files
#!git clone https://github.com/anaustinbeing/haar-cascade-files.git

In [ ]:
import cv2
from PIL import Image
import numpy as np
import shutil
import os
from datetime import datetime

try:
  os.makedirs("images")
except:
  print("The folder already exist")

def rotate(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image
    (h, w) = image.shape[:2]

    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        center = (w // 2, h // 2)

    # perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    # return the rotated image
    return rotated

def  findLargestBB(bbs):
  areas = [w*h for x,y,w,h in bbs]
  if not areas:
      return False, None
  else:
      i_biggest = np.argmax(areas) 
      biggest = bbs[i_biggest]
      return True, biggest

cap = cv2.VideoCapture("mask-detector-covid/data/video.mp4")

if (cap.isOpened()== False): 
  print("Error opening video stream or file")

model_face = cv2.CascadeClassifier('mask-detector-covid/models/haar-cascade-files/haarcascade_frontalface_default.xml')

while(cap.isOpened()):

  # lettura immagine
  ret, frame = cap.read()
  
  # coversione immagine da BGR a RGB
  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
  #frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
  
  # Rotazione dell'immagine
  frame = rotate(frame, -90)

  # Trova tutte le facce nell'immagine
  faces = model_face.detectMultiScale(frame,scaleFactor=1.05,minNeighbors=4, flags=cv2.CASCADE_DO_ROUGH_SEARCH | cv2.CASCADE_SCALE_IMAGE)
  
  # Trova la faccia piu grande (area piu grande)
  ret, facebig = findLargestBB(faces)

  # Per ogni faccia fai qualcosa
  if ret == True:

    # Extra coordiante of largest image
    x,y,w,h = facebig
    
    # Crop image 
    roi = frame[y:y+h,x:x+w]

    date = datetime.now()
    date = date.strftime("%d-%m-%Y_%H-%M-%S")
    roi = cv2.cvtColor(roi, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join("images",date+".png"), roi)
    # Disegna il quadrato
    #cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)

    # Solo sul vostro pc
    #cv2.imshow("Image", roi)
    #cv2.waitKey(33)

    # In colab o jupyter
    #frame = cv2.resize(frame, (128,128))
    #im_pil = Image.fromarray(roi)
    #display(im_pil)


The folder already exist


## Test haar cascade

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import cv2
from PIL import Image

im = cv2.imread("/content/photo.jpg",1) # 0 grayscale and 1 color
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) # BGR --> RGB
gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY) # BGR --> RGB

#im = cv2.resize(im, (64,64))

# load a model
cl = cv2.CascadeClassifier("/content/covid-mask-classifier/models/haarcascade/haarcascade_frontalface_default.xml")

faces = cl.detectMultiScale( gray, scaleFactor=1.05, minNeighbors=5,minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
for (x, y, w, h) in faces:
  
  # Process
  im_p = im.copy()
  roi = im_p[y:y+h,x:x+w]


  cv2.rectangle(im, (x,y), (x+w,y+h) ,(0, 255, 0), 2)

  display( Image.fromarray(roi) ) # RGB

# cv2.IMREAD_GRAYSCALE, cv2.IMREAD_COLOR, cv2.IMREAD_UNCHAGED
# --> 0 - 255 --_> 215 x 215

display( Image.fromarray(im) ) # RGB

im = cv2.cvtColor(im, cv2.COLOR_RGB2BGR) # RGB --> BGR
cv2.imwrite("manuel.png",im) # BGR

## OS Shutil 

In [ ]:
import os
import shutil
folder_path = os.path.join("mask-detector-covid", "dataset", "mask")
names = os.listdir(folder_path)
for name in names:
  im = cv2.imread( os.path.join(folder_path, name) ,1 )
  display(Image.fromarray(im))
  break

try:
  os.makedirs("test")
except:
  print("I am unable to do this thing")
shutil.rmtree("test")

In [ ]:
from datetime import datetime
date = datetime.now()
date.strftime("%d-%m-%Y_%H-%M-%S")